In [1]:
include("./src/methods.jl")
# include("./src/auxfunctions.jl")

circussimplex (generic function with 1 method)

In [2]:
using PyPlot, GLPK, JuMP, Clp

In [176]:

"""
    simplex_format(c, A, b)
Adjust the format from
           min  dot(c, x)
    subject to  A x ≦ b
To the standard Simplex fomat which is
           min  dot(c, x)
    subject to  A x = b
"""

function simplex_format(c,A,b)
    row_remove = []
    var_stay   = []
    for i = 1:size(A)[1]
        present_variables = .!(A[i,:].≈0)
        if sum(present_variables) == 1
            if all(A[i,present_variables].<0) && b[i].≈0
                push!(row_remove,i)
                push!(var_stay,findmax(present_variables)[2])
            elseif all(sign.(A[i,present_variables]).!=sign(b[i]))
                push!(row_remove,i)
                push!(var_stay,findmax(present_variables)[2])
            else
                v = zeros(size(A)[1])
                v[i] = 1
                A = hcat(A,v)
                c = vcat(c,0)
                push!(var_stay,findmax(present_variables)[2])
                push!(var_stay,size(c)[1])
            end
        else
            v = zeros(size(A)[1])
            v[i] = 1
            A = hcat(A,v)
            c = vcat(c,0)
            push!(var_stay,size(c)[1])
        end
    end

    A = A[setdiff(1:end, row_remove),:]
    b = b[setdiff(1:end, row_remove),:]
    var_split = setdiff(1:size(c)[1],var_stay)
    aux = 0
    for i in var_split
        if i == size(A[2])
            A = hcat(A,-A[:,i])
            c = hcat(c,-c[:,i])
        else
            A_ = A[:,1:i+aux]
            A_ = hcat(A_,-A[:,i+aux])
            _A = A[:,i+1+aux:end]
            A  = hcat(A_,_A)

            c_ = c[1:i+aux]
            c_ = vcat(c_,-c[i+aux])
            _c = c[i+1+aux:end]
            c  = vcat(c_,_c)
            aux = aux + 1
        end
    end
    
    return c,A,b,var_split
end

simplex_format (generic function with 1 method)

In [177]:
function variables_relation(varsplit,n)
    simplex_to_circus = Dict()
    k = 0
    for i in 1:n
        if i in varsplit
            simplex_to_circus[i] = [i+k,i+1+k]
            k += 1 
        else
            simplex_to_circus[i] = [i]
        end
    end
    return simplex_to_circus
end

variables_relation (generic function with 1 method)

In [178]:
function variable_simplex_to_circus(simplex_to_circus, x_simplex)
    n = size(collect(keys(simplex_to_circus)))[1]
    x_circus = zeros(n)
    for (k,i) in simplex_to_circus
        if size(i)[1] > 1
            x_circus[k] = x_simplex[i[1]] - x_simplex[i[2]]
        else
            x_circus[k] = x_simplex[i[1]]
        end
    end
    return x_circus
end

variable_simplex_to_circus (generic function with 2 methods)

In [179]:
function variable_circus_to_simplex(simplex_to_circus,x_circus,
        c_simplex,A_simplex,b_simplex)
    x_simplex = zeros(size(c_simplex)[1])
    n_simplex = 0
    for (k,i) in simplex_to_circus
        if size(i)[1] > 1
            n_simplex+=2
            if x_circus[k] < 0
                x_simplex[i[1]] = 0
                x_simplex[i[2]] = -x_circus[k]
            else
                x_simplex[i[1]] = x_circus[k]
                x_simplex[i[2]] = 0
            end
        else
            n_simplex+=1
            x_simplex[i[1]] = x_circus[k]
        end
    end

    if n_simplex < size(c_simplex)[1]
        x_simplex[n_simplex+1:size(cs)[1]] = A_simplex*x_simplex - b_simplex
    end
    return x_simplex
end

variable_circus_to_simplex (generic function with 1 method)

In [180]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
cs,As,bs,varsplit = simplex_format(c,A,b)
x = [0.,0,0,0,0,0,0,4]
x_simplex = SimplexFromBFS(cs,As,bs,x,index_bfs=[6,7,8],index_nfs=[1,2,3,4,5])
x_simplex = x_simplex[:,2]

5   6
1   8
3   7


8-element Array{Float64,1}:
 4.0
 0.0
 0.0
 0.0
 4.0
 0.0
 0.0
 0.0

In [181]:
simplex_to_circus = variables_relation(varsplit,3)
x_circus = variable_simplex_to_circus(simplex_to_circus,x_simplex)

3-element Array{Float64,1}:
  4.0
  0.0
 -4.0

In [182]:
x_simplex = variable_circus_to_simplex(simplex_to_circus,x_circus,cs,As,bs)

8-element Array{Float64,1}:
 4.0
 0.0
 0.0
 0.0
 4.0
 0.0
 0.0
 0.0

#### Phase 1


In [275]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,-1.]

x = [0,0,0,0,0,0,0,4,1]
cs,As,bs,varsplit = simplex_format(c,A,b)

([0, 0, 0, 1, -1, 0, 0, 0, 0], [-1.0 1.0 … 0.0 0.0; -1.0 -1.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; -1.0 0.0 … 0.0 1.0], [0.0; 0.0; 4.0; -1.0], [2, 3])

In [256]:
function phaseI_simplex_problem(A,b)
    for i = 1:size(A)[1]
        if b[i]<0
            b[i] = -b[i]
            A[i,:] = -A[i,:]
        end
    end
    A = hcat(A,Matrix{Float64}(I, size(A)[1], size(A)[1]))
    c = zeros(size(A)[2])
    c[end-size(A)[1]+1:end] .= 1
    x = zeros(size(A)[2])
    x[end-size(A)[1]+1:end] = b
    return c,A,b,x
end

phaseI_simplex_problem (generic function with 1 method)

In [262]:
cpi, Api, bpi, xpi = phaseI_simplex_problem(As,bs)
xpi

13-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 4.0
 1.0

In [267]:
xpi[13-3:13]

4-element Array{Float64,1}:
 0.0
 0.0
 4.0
 1.0

In [282]:
x_simplex = SimplexFromBFS(cpi,Api,bpi,xpi,index_bfs=[10,11,12,13]
    ,index_nfs=[1,2,3,4,5,6,7,8,9])

x_simplex = x_simplex[1:9,2]

simplex_to_circus = variables_relation(varsplit,3)
x_circus = variable_simplex_to_circus(simplex_to_circus,x_simplex)

5   10
1   13
3   11
8   12


3-element Array{Float64,1}:
  1.0
  0.0
 -1.0

In [293]:
As*x_simplex

4-element Array{Float64,1}:
  0.0
  0.0
  4.0
 -1.0

In [294]:
x_circus

3-element Array{Float64,1}:
  1.0
  0.0
 -1.0

In [33]:
c = [0,0,1]
A = [-1. 1. -1.;
     -1. -1. -1.;
     1. 0. 0.
     -1. 0. 0.]
b = [0,0,4.,0.]
xo = [1,-1,0.0]

pygui(true)
px= [0,4 ,4 ,0 ,0,0,0 ,0,4,4,4 ,4]
py= [0,0 ,-4,-4,0,4,-4,4,4,-4,0,4]
pz= [0,-4,0 ,4 ,0,4,4 ,4,0,0,-4,0]
plt= PyPlot.plot3D(px,py,pz)
PyPlot.scatter3D([xo[1]],[xo[2]],[xo[3]])

PyObject <mpl_toolkits.mplot3d.art3d.Path3DCollection object at 0x7f4130524eb0>

In [179]:
model = Model(
    with_optimizer(Clp.Optimizer,
        SolveType = 1, MaximumIterations=2, PresolveType=1)
)
@variable(model, x[1:size(c)[1]])
set_start_value(x[1],4.0)
set_start_value(x[2],0.0)
set_start_value(x[3],-4.0)
@constraint(model, A * x .<= b)
@objective(model, Min, c' * x)

optimize!(model)

@show objective_value(model);
for i = 1:size(c)[1]
    @show value(x[i]);
end

termination_status(model)

objective_value(model) = -1.0e-12
value(x[i]) = 0.0
value(x[i]) = 1.0e-12
value(x[i]) = -1.0e-12
Clp0006I 0  Obj 0 Dual inf 0.0099999 (1)
Clp0029I End of values pass after 1 iterations
Clp0006I 1  Obj 0 Dual inf 0.0199998 (2) w.o. free dual inf (0)
Clp0003I Stopped - objective value -1e-12
Clp0032I Stopped objective -1e-12 - 2 iterations time 0.002


┌ Warning: MathOptInterface.VariablePrimalStart() is not supported by MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{Clp.Optimizer,MathOptInterface.Utilities.UniversalFallback{MathOptInterface.Utilities.Model{Float64}}}}. This information will be discarded.
└ @ MathOptInterface.Utilities /home/davi/.julia/packages/MathOptInterface/RmalA/src/Utilities/copy.jl:140


OTHER_LIMIT::TerminationStatusCode = 18

In [166]:
termination_status(model) == MOI.OTHER_LIMIT
typeof(MOI.OTHER_LIMIT)

Enum MathOptInterface.TerminationStatusCode:
OPTIMIZE_NOT_CALLED = 0
OPTIMAL = 1
INFEASIBLE = 2
DUAL_INFEASIBLE = 3
LOCALLY_SOLVED = 4
LOCALLY_INFEASIBLE = 5
INFEASIBLE_OR_UNBOUNDED = 6
ALMOST_OPTIMAL = 7
ALMOST_INFEASIBLE = 8
ALMOST_DUAL_INFEASIBLE = 9
ALMOST_LOCALLY_SOLVED = 10
ITERATION_LIMIT = 11
TIME_LIMIT = 12
NODE_LIMIT = 13
SOLUTION_LIMIT = 14
MEMORY_LIMIT = 15
OBJECTIVE_LIMIT = 16
NORM_LIMIT = 17
OTHER_LIMIT = 18
SLOW_PROGRESS = 19
NUMERICAL_ERROR = 20
INVALID_MODEL = 21
INVALID_OPTION = 22
INTERRUPTED = 23
OTHER_ERROR = 24

In [115]:
vector_model = Model(
    with_optimizer(GLPK.Optimizer, it_lim = 1)
)

@variable(vector_model, x[1:size(c)[1]])
@constraint(vector_model, A * x .<= b)
@objective(vector_model, Min, c' * x)

optimize!(vector_model)

@show objective_value(vector_model);
for i = 1:size(c)[1]
    @show value(x[i]);
end
termination_status(vector_model)

objective_value(vector_model) = 0.0
value(x[i]) = 0.0
value(x[i]) = 0.0
value(x[i]) = 0.0


OPTIMIZE_NOT_CALLED::TerminationStatusCode = 0

In [123]:
display(typeof(MOI.OPTIMAL))

Enum MathOptInterface.TerminationStatusCode:
OPTIMIZE_NOT_CALLED = 0
OPTIMAL = 1
INFEASIBLE = 2
DUAL_INFEASIBLE = 3
LOCALLY_SOLVED = 4
LOCALLY_INFEASIBLE = 5
INFEASIBLE_OR_UNBOUNDED = 6
ALMOST_OPTIMAL = 7
ALMOST_INFEASIBLE = 8
ALMOST_DUAL_INFEASIBLE = 9
ALMOST_LOCALLY_SOLVED = 10
ITERATION_LIMIT = 11
TIME_LIMIT = 12
NODE_LIMIT = 13
SOLUTION_LIMIT = 14
MEMORY_LIMIT = 15
OBJECTIVE_LIMIT = 16
NORM_LIMIT = 17
OTHER_LIMIT = 18
SLOW_PROGRESS = 19
NUMERICAL_ERROR = 20
INVALID_MODEL = 21
INVALID_OPTION = 22
INTERRUPTED = 23
OTHER_ERROR = 24

In [136]:
display(typeof(MOI.FEASIBLE_POINT))

Enum MathOptInterface.ResultStatusCode:
NO_SOLUTION = 0
FEASIBLE_POINT = 1
NEARLY_FEASIBLE_POINT = 2
INFEASIBLE_POINT = 3
INFEASIBILITY_CERTIFICATE = 4
NEARLY_INFEASIBILITY_CERTIFICATE = 5
REDUCTION_CERTIFICATE = 6
NEARLY_REDUCTION_CERTIFICATE = 7
UNKNOWN_RESULT_STATUS = 8
OTHER_RESULT_STATUS = 9

In [10]:
x = xo
num_const, num_var = size(A)
cnormed = c/norm(c)
Anormed = copy(A)
bnormed = copy(b)
for  i in 1:num_const
    normA = norm(Anormed[i,:])
    Anormed[i,:] /= normA
    bnormed[i] /= normA
end
sortedJ = sortperm((Anormed*cnormed),rev=true)
J = findall(abs.(b-A*x) .< 1e-8)
refinesolution(xo, A, b, c, num_var, 1e-8)

3-element Array{Float64,1}:
  4.0
  1.1102230246251585e-16
 -3.999999999999999

3-element Array{Float64,1}:
  1.0
 -1.0
  0.0